In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from modelcreate.modelcreate import *
import tensorflow as tf
from tensorflow import keras
import numpy as np
import yolo_Mobilenet
import logging
import model_Mobilenet
logger=tf.get_logger()
logger.setLevel(logging.ERROR)

In [2]:
yolomodel=createyolo()
yolomodel.load_weights('outputmodel/yolomodel/yoloweight.h5',by_name=True)
#yolo모델생성

Create YOLOv3 model with 9 anchors and 1 classes.
[<tf.Tensor 'conv2d_6/BiasAdd:0' shape=(None, None, None, 18) dtype=float32>, <tf.Tensor 'conv2d_14/BiasAdd:0' shape=(None, None, None, 18) dtype=float32>, <tf.Tensor 'conv2d_22/BiasAdd:0' shape=(None, None, None, 18) dtype=float32>]
a


In [3]:
crnnmodel=crnn()
crnnmodel.load_weights('outputmodel/crnnmodel/crnnweight.h5')
#crnn모델생성

In [69]:
import cv2
import copy
imgw=320
imgh=320

img=cv2.imread('D:/engocr/yolo/YOLOv3/test_images/test_images/'+'1d01568a9f8b528f.jpg')
img=img.astype(np.float32)
img=cv2.resize(img,(imgw,imgh))

cv2.imwrite('abc.jpg',img)
img=img/255.0
img=np.expand_dims(img,0)


#yolo모델삽입

In [39]:
features=yolomodel.predict(img)
featureslen=len(features)
#yolo모델예측


In [48]:
def sigmoid(x):
    return (1./(1.+np.exp(-x)))+0.2
num_classes=1
sigmoid = np.vectorize(sigmoid)
proto_box=[]
proto_scores=[]
for idx, val in enumerate(features):
    anchors = np.array([np.array([10,13]), np.array([16,30]), np.array([33,23]), np.array([30,61]), np.array([62,45]), np.array([59,119]), np.array([116,90]), np.array([156,198]), np.array([373,326])])
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]]
    input_shape = np.asarray(np.shape(features[0])[1 : 3]) * 32
    first = anchors[anchor_mask[idx]]
    image_size = (320, 320)
    num_anchors = len(first)
    anchors_tensor = np.reshape(first, [1, 1, 1, num_anchors, 2])
    grid_shape = np.shape(val)[1 : 3]
    b = np.reshape(np.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1])
    grid_y = np.tile(np.reshape(np.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]), [1, grid_shape[1], 1, 1])
    grid_x = np.tile(np.reshape(np.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),
                    [grid_shape[0], 1, 1, 1])
    grid = np.concatenate([grid_x, grid_y], axis=3)
    feats = np.reshape(
            val, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    box_xy = (sigmoid(feats[..., :2]) + grid) / grid_shape[::-1]
    pre_box_wh = feats[..., 2:4] * anchors_tensor / input_shape[::-1]
    box_wh = np.exp(feats[..., 2:4]) * anchors_tensor / input_shape[::-1]
    box_confidence = sigmoid(feats[..., 4:5])
    box_class_probs = sigmoid(feats[..., 5:])

    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    image_shape = np.array([320, 320])
    new_shape = np.round((image_shape * np.min(input_shape/image_shape)))
    offset = (input_shape-new_shape)/2./input_shape
    scale = input_shape/new_shape
    box_yx = (box_yx - offset) #* scale
    #box_hw *= scale
    #box_hw*=1.0

    box_mins = box_yx - (box_hw / 2.)
    box_maxes = box_yx + (box_hw / 2.)

    boxes = np.concatenate([
            box_mins[..., 0:1],  # y_min
            box_mins[..., 1:2],  # x_min
            box_maxes[..., 0:1],  # y_max
            box_maxes[..., 1:2]  # x_max
        ], axis=4)

    # Scale boxes back to original image shape.
    scaler = np.concatenate([image_shape, image_shape])
    boxes *= scaler
    #here at original implementation is loosing of data, because batch size is ignored
    boxes = np.reshape(boxes, [boxes.shape[0], -1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = np.reshape(box_scores, [box_scores.shape[0], -1, num_classes])
    proto_box.append(boxes)
    proto_scores.append(box_scores)
proto_box = np.concatenate(proto_box, axis=1)
proto_scores = np.concatenate(proto_scores, axis=1)
mask = proto_scores >= 0.6
_boxes = []
#yolo모델후처리(함수로빼자)

In [49]:
for idx, batch in enumerate(proto_scores):
    final_classes = []
    final_boxes = []
    final_scores = []
    for c in range(num_classes):
        class_boxes = proto_box[idx, mask[idx, :, c]]
        class_box_scores = proto_scores[idx, :, c][mask[idx, :, c]]
        classes = np.ones_like(class_box_scores, dtype="int32") * c
        final_boxes.append(class_boxes)
        final_scores.append(class_box_scores)
    final_boxes = np.concatenate(final_boxes, axis=0)
    final_scores = np.concatenate(final_scores, axis=0)
    _boxes.append(final_boxes)
#yolo모델후처리(함수로빼자 위랑묶어서)

In [55]:
for bbbox in final_boxes:
    print(str(bbbox[1])+' '+str(bbbox[0])+' '+str(bbbox[3])+' '+str(bbbox[2]))

154.08139461954963 102.65595813546503 278.54511637171635 187.68931706700647
166.3247330568517 261.4013620206501 245.5788777254308 313.8322128841068
180.81008804147154 263.6151238279629 254.48471844499016 312.42889781764035


In [80]:
#cropped_img = imgcopy[int(final_boxes[0][1]):  int(final_boxes[0][3]),int(final_boxes[0][0]): int(final_boxes[0][2])]
aaimg=cv2.imread('D:/engocr/yolo/YOLOv3/test_images/test_images/'+'1d01568a9f8b528f.jpg')
aaimg=cv2.resize(aaimg,(imgw,imgh))
for cropbox in final_boxes:
    cropped_img = aaimg[int(cropbox[0]):  int(cropbox[2]),int(cropbox[0]): int(cropbox[3])]
#cropped_img = aaimg[261:313,166:245]
cv2.imshow('ada',cropped_img)
cv2.waitKey()
cv2.destroyAllWindows()

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-uzca7qz1\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [19]:
type(features)

list

In [48]:
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
 
    classIds = []
    confidences = []
    boxes = []
    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        for det in out:
            for detection in det :
                scores = detection[4:5]
                classId = np.argmax(scores)
                print(len(scores[2]))
                confidence = scores[classId-1]
                if confidence > confThreshold:
                    center_x = int(detection[0] * frameWidth)
                    center_y = int(detection[1] * frameHeight)
                    width = int(detection[2] * frameWidth)
                    height = int(detection[3] * frameHeight)
                    left = int(center_x - width / 2)
                    top = int(center_y - height / 2)
                    classIds.append(classId)
                    confidences.append(float(confidence))
                    boxes.append([left, top, width, height])
 
    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    #indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
      #  drawPred(classIds[i], confidences[i], left, top, left + width, top + height)
    return boxes

In [49]:
ret=postprocess(imgcopy,features)

IndexError: index 2 is out of bounds for axis 0 with size 1

In [45]:
for i, c in reversed(list(enumerate(classes))):
    box=boxes[i]
    score=scores[i]
    
    #y1,x1,y2,x2=box
    print(len(c))
    break;

40


In [14]:
output=yolomodel.layers[-4].output
#yolopremodel=keras.models.Model(inputs=yolomodel.input,outputs=output)

In [59]:
yolomodel.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 3 128         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu (ReLU)               (None, None, None, 3 0           conv1_bn[0][0]                   
____________________________________________________________________________________________

In [11]:
def yolo_boxes_and_scores(feats, anchors, num_classes, input_shape, image_shape):
    '''Process Conv layer output'''
    box_xy, box_wh, box_confidence, box_class_probs = yolo_head(feats,
        anchors, num_classes, input_shape)
    boxes = yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = K.reshape(boxes, [-1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = K.reshape(box_scores, [-1, num_classes])
    return boxes, box_scores


def yolo_eval(yolo_outputs,
              anchors,
              num_classes,
              image_shape,
              max_boxes=20,
              score_threshold=.6,
              iou_threshold=.5):
    """Evaluate YOLO model on given input and return filtered boxes."""

    num_layers = len(yolo_outputs)

    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]] # default setting
    input_shape = K.shape(yolo_outputs[0])[1:3] * 32

    # print("yolo_outputs",yolo_outputs)
    boxes = []
    box_scores = []
    for l in range(num_layers):
        _boxes, _box_scores = yolo_boxes_and_scores(yolo_outputs[l],
            anchors[anchor_mask[l]], num_classes, input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = K.concatenate(boxes, axis=0)
    box_scores = K.concatenate(box_scores, axis=0)

    mask = box_scores >= score_threshold
    max_boxes_tensor = K.constant(max_boxes, dtype='int32')
    boxes_ = []
    scores_ = []
    classes_ = []
    for c in range(num_classes):
        # TODO: use keras backend instead of tf.
        class_boxes = tf.boolean_mask(boxes, mask[:, c])
        class_box_scores = tf.boolean_mask(box_scores[:, c], mask[:, c])
        nms_index = tf.image.non_max_suppression(
            class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=iou_threshold)
        class_boxes = K.gather(class_boxes, nms_index)
        class_box_scores = K.gather(class_box_scores, nms_index)
        classes = K.ones_like(class_box_scores, 'int32') * c
        boxes_.append(class_boxes)
        scores_.append(class_box_scores)
        classes_.append(classes)
    boxes_ = K.concatenate(boxes_, axis=0)
    scores_ = K.concatenate(scores_, axis=0)
    classes_ = K.concatenate(classes_, axis=0)

    return boxes_, scores_, classes_
def _get_anchors(anchors_path):
    anchors_path = os.path.expanduser(anchors_path)
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)
def yolo_head(feats, anchors, num_classes, input_shape, calc_loss=False):
    """Convert final layer features to bounding box parameters."""
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params.
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    grid_shape = K.shape(feats)[1:3] # height, width
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]),
        [1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),
        [grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, K.dtype(feats))

    feats = K.reshape(
        feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # Adjust preditions to each spatial grid point and anchor size.
    box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[...,::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[...,::-1], K.dtype(feats))
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.sigmoid(feats[..., 5:])

    if calc_loss == True:
        return grid, feats, box_xy, box_wh
    return box_xy, box_wh, box_confidence, box_class_probs

In [12]:
anchors_path = 'model_data/yolo_anchors.txt'
classes_path = 'model_data/car_classes.txt'
anchors = _get_anchors(anchors_path)
len(anchors)

9

In [13]:
t=tf.convert_to_tensor(value=features)
boxes, scores, classes = model_Mobilenet.yolo_eval(features,anchors,1,K.placeholder(shape=(2, )))

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\tensor_util.py:458: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  nparray = np.array(values, dtype=np_dt)


ValueError: could not broadcast input array from shape (10,10,18) into shape (1)

In [62]:
yyyy.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv1 (Conv2D)                  (None, None, None, 3 864         input_5[0][0]                    
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 3 128         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu (ReLU)               (None, None, None, 3 0           conv1_bn[0][0]                   
__________________________________________________________________________________________________
conv_dw_1 (DepthwiseConv2D)     (None, None, None, 3 288         conv1_relu[0][0]                 
______________________________________________________________________________________________

In [3]:
yolotest.scores

<tf.Tensor 'concat_12/concat:0' shape=(None,) dtype=float32>

In [4]:
yolo_Mobilenet.detect_img(yolotest,'D:/engocr/yolo/YOLOv3/test_images/test_images/00ef00c7ed62f6d6.jpg')

FailedPreconditionError: 2 root error(s) found.
  (0) Failed precondition: Could not find variable conv2d_12/kernel. This could mean that the variable has been deleted. In TF1, it can also mean the variable is uninitialized. Debug info: container=localhost, status=Not found: Container localhost does not exist. (Could not find resource: localhost/conv2d_12/kernel)
	 [[node conv2d_12/Conv2D/ReadVariableOp (defined at c:\users\user\appdata\local\programs\python\python36\lib\site-packages\keras\layers\convolutional.py:245) ]]
	 [[concat_11/concat/_55]]
  (1) Failed precondition: Could not find variable conv2d_12/kernel. This could mean that the variable has been deleted. In TF1, it can also mean the variable is uninitialized. Debug info: container=localhost, status=Not found: Container localhost does not exist. (Could not find resource: localhost/conv2d_12/kernel)
	 [[node conv2d_12/Conv2D/ReadVariableOp (defined at c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ker

In [ ]:
#D:/engocr/yolo/YOLOv3/test_images/test_images/00ef00c7ed62f6d6.jpg